In [4]:
import time
import gc
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')
from preprocessing import *
from plotting import *

In [6]:
df_db = group_datafiles_byID('../datasets/preprocessed/HT_Sensor_prep_metadata.dat', '../datasets/preprocessed/HT_Sensor_prep_dataset.dat')
df_db = reclassify_series_samples(df_db)

In [7]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

features_id = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'Temp.', 'Humidity', 'id']
features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

class_dict = {
    'background':0,
    'banana':1,
    'wine':2
}

def class_to_int(y):
    return np.array([class_dict[x] for x in y.tolist()])

def get_majority_class(y):
    (values, counts) = np.unique(y,return_counts=True)
    ind = np.argmax(counts)
    return values[ind]

def separate_ids(df_db):
    X = df_db[features_id].values
    y = df_db['class'].values
    y = class_to_int(y)
    ids = np.unique(X[:,-1])
    new_X = []
    new_y = []
    for id in ids:
        idx = np.where(X[:,-1] == id)
        new_X.append(X[idx][:,features])
        new_y.append(y[idx])
    return new_X, new_y

def generate_sequences(X, y, seq_len, step):
    new_X = []
    new_y = []
    for i in range(0, X.shape[0] - seq_len, step):
        new_X.append( X[i: i + seq_len] )
        new_y.append( get_majority_class(y[i: i + seq_len]) )
    num_seqs = len(new_X)
    new_X = np.array(new_X)
    new_y = to_categorical(new_y, num_classes=3)
    return new_X, new_y, num_seqs

def raw_windows(X, y, seq_len, step):
    new_X = []
    new_y = []
    total_seqs = 0
    for id in range(len(X)):
        X_id, y_id, num_seqs = generate_sequences(X[id], y[id], seq_len, step)
        new_X.append(X_id)
        new_y.append(y_id)
        total_seqs += num_seqs
    return new_X, new_y, total_seqs

def get_train_test_split(X, y, test_size=0.33, concatenate_train=True, cnn=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    if concatenate_train:
        X_train = np.concatenate(X_train)
        y_train = np.concatenate(y_train)
        if cnn:
            X_train = np.reshape(X_train, (-1,seq_len,10,1))
    X_test = np.concatenate(X_test)
    y_test = np.concatenate(y_test)
    if cnn:
        X_test = np.reshape(X_test, (-1,seq_len,10,1))
    return X_train, X_test, y_train, y_test

In [8]:
seq_len = 120
step = 1

X, y = separate_ids(df_db)
X, y, total_seqs = raw_windows(X, y, seq_len, step)

In [9]:
from tensorflow.keras.layers import Dense, GRU, LSTM, Input, Conv2D, MaxPooling2D, Flatten, GaussianNoise
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

opti = Adam()
er = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=4,
                   restore_best_weights=True,
                   verbose=0,
                   mode='auto')

def get_cnn_model(kernel_size, pool_size, n_units):
    model = Sequential()
    model.add(Conv2D(n_units, kernel_size, activation='relu', input_shape=(seq_len, 10, 1), padding='same'))
    model.add(MaxPooling2D(pool_size))
    model.add(Conv2D(n_units, kernel_size, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size))
    model.add(Conv2D(n_units, kernel_size, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size))
    model.add(Flatten())
    model.add(Dense(n_units, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
              optimizer=opti,
              metrics = ['accuracy'])
    model.summary()
    return model

def get_rnn_model(n_units, seq_to_seq=False, lstm_gru='LSTM'):
    if lstm_gru == 'LSTM':
        Recurrent = LSTM
    else:
        Recurrent = GRU
    input = Input(shape = (seq_len,10))
    if seq_to_seq:
        x = Recurrent(n_units, return_sequences=True)(input)
        x = Recurrent(n_units)(x)
    else:
        x = Recurrent(n_units)(input)
    output = Dense(3, activation = 'softmax')(x)
    model = Model(input, output)
    model.compile(loss='categorical_crossentropy',
              optimizer=opti,
              metrics = ['accuracy'])
    model.summary()
    return model

In [10]:
X_train, X_test, y_train, y_test = get_train_test_split(X, y, concatenate_train=False)

In [11]:
epochs=30
batch_size=512
train_val_sets = [get_train_test_split(X_train, y_train, test_size=0.2) for _ in range(5)]

In [12]:
rec = ['LSTM', 'GRU']
n_units = [4,8,16]
seq_to_seq = [True, False]

histories = []

for x in rec:
    for y in n_units:
        for z in seq_to_seq:
            for bag in train_val_sets:
                model = get_rnn_model(y, z, x)
                X_train_, X_val, y_train_, y_val = bag
                h = model.fit(X_train_, y_train_, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_val, y_val), callbacks=[er])
                histories.append(h)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 10)]         0         
_________________________________________________________________
lstm (LSTM)                  (None, 120, 4)            240       
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 144       
_________________________________________________________________
dense (Dense)                (None, 3)                 15        
Total params: 399
Trainable params: 399
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
964/964 [==============================] - 122s 127ms/step - loss: 0.6317 - accuracy: 0.7680 - val_loss: 0.5311 - val_accuracy: 0.8328
Epoch 2/30
964/964 [==============================] - 119s 124ms/step - loss: 0.5172 - accuracy: 0.7973 - va

938/938 [==============================] - 186s 198ms/step - loss: 0.4459 - accuracy: 0.8308 - val_loss: 0.4737 - val_accuracy: 0.8293
Epoch 8/30
938/938 [==============================] - 183s 195ms/step - loss: 0.4369 - accuracy: 0.8380 - val_loss: 0.5737 - val_accuracy: 0.7473
Epoch 9/30
938/938 [==============================] - 188s 200ms/step - loss: 0.4355 - accuracy: 0.8392 - val_loss: 0.5286 - val_accuracy: 0.7603
Epoch 10/30
938/938 [==============================] - 184s 197ms/step - loss: 0.4272 - accuracy: 0.8438 - val_loss: 0.5633 - val_accuracy: 0.7698
Epoch 11/30
938/938 [==============================] - 185s 198ms/step - loss: 0.4612 - accuracy: 0.8254 - val_loss: 0.5457 - val_accuracy: 0.8131
Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 120, 10)]         0         
___________________________________________________________

Epoch 1/30
938/938 [==============================] - 109s 116ms/step - loss: 0.5754 - accuracy: 0.7738 - val_loss: 0.5847 - val_accuracy: 0.8181
Epoch 2/30
938/938 [==============================] - 102s 109ms/step - loss: 0.5046 - accuracy: 0.7962 - val_loss: 0.5948 - val_accuracy: 0.7990
Epoch 3/30
938/938 [==============================] - 66s 71ms/step - loss: 0.4752 - accuracy: 0.8002 - val_loss: 0.5954 - val_accuracy: 0.7820
Epoch 4/30
938/938 [==============================] - 69s 73ms/step - loss: 0.4609 - accuracy: 0.8042 - val_loss: 0.5904 - val_accuracy: 0.7822
Epoch 5/30
938/938 [==============================] - 47s 50ms/step - loss: 0.4466 - accuracy: 0.8104 - val_loss: 0.5718 - val_accuracy: 0.7955
Epoch 6/30
938/938 [==============================] - 47s 51ms/step - loss: 0.4400 - accuracy: 0.8158 - val_loss: 0.6665 - val_accuracy: 0.7538
Epoch 7/30
938/938 [==============================] - 47s 50ms/step - loss: 0.4396 - accuracy: 0.8157 - val_loss: 0.5810 - val_accur

Epoch 1/30
958/958 [==============================] - 113s 118ms/step - loss: 0.6176 - accuracy: 0.7469 - val_loss: 0.6334 - val_accuracy: 0.7798
Epoch 2/30
958/958 [==============================] - 135s 141ms/step - loss: 0.6050 - accuracy: 0.7414 - val_loss: 0.7145 - val_accuracy: 0.7066
Epoch 3/30
958/958 [==============================] - 122s 127ms/step - loss: 0.5259 - accuracy: 0.7715 - val_loss: 0.7074 - val_accuracy: 0.7436
Epoch 4/30
958/958 [==============================] - 140s 146ms/step - loss: 0.5362 - accuracy: 0.7922 - val_loss: 0.6318 - val_accuracy: 0.7374
Epoch 5/30
958/958 [==============================] - 120s 126ms/step - loss: 0.4708 - accuracy: 0.8192 - val_loss: 0.6094 - val_accuracy: 0.7613
Epoch 6/30
958/958 [==============================] - 118s 123ms/step - loss: 0.4436 - accuracy: 0.8282 - val_loss: 0.6195 - val_accuracy: 0.7521
Epoch 7/30
958/958 [==============================] - 187s 195ms/step - loss: 0.4207 - accuracy: 0.8448 - val_loss: 0.6624 -

Epoch 1/30
958/958 [==============================] - 89s 93ms/step - loss: 0.6681 - accuracy: 0.7378 - val_loss: 0.5996 - val_accuracy: 0.7888
Epoch 2/30
958/958 [==============================] - 117s 123ms/step - loss: 0.6093 - accuracy: 0.7504 - val_loss: 0.5903 - val_accuracy: 0.8061
Epoch 3/30
958/958 [==============================] - 117s 122ms/step - loss: 0.5774 - accuracy: 0.7707 - val_loss: 0.6041 - val_accuracy: 0.8041
Epoch 4/30
958/958 [==============================] - 118s 123ms/step - loss: 0.5608 - accuracy: 0.7770 - val_loss: 0.6180 - val_accuracy: 0.8032
Epoch 5/30
958/958 [==============================] - 118s 123ms/step - loss: 0.5512 - accuracy: 0.7798 - val_loss: 0.6279 - val_accuracy: 0.7972
Epoch 6/30
958/958 [==============================] - 117s 122ms/step - loss: 0.5384 - accuracy: 0.7832 - val_loss: 0.6123 - val_accuracy: 0.8050
Model: "functional_37"
_________________________________________________________________
Layer (type)                 Output S

...

964/964 [==============================] - 146s 152ms/step - loss: 0.3590 - accuracy: 0.8729 - val_loss: 0.6661 - val_accuracy: 0.7676
Model: "functional_83"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_42 (InputLayer)        [(None, 120, 10)]         0         
_________________________________________________________________
gru_17 (GRU)                 (None, 120, 8)            480       
_________________________________________________________________
gru_18 (GRU)                 (None, 8)                 432       
_________________________________________________________________
dense_41 (Dense)             (None, 3)                 27        
Total params: 939
Trainable params: 939
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
940/940 [==============================] - 148s 158ms/step - loss: 0.6235 - accuracy: 0.7598 - val_loss: 0.

931/931 [==============================] - 120s 129ms/step - loss: 0.3750 - accuracy: 0.8670 - val_loss: 0.3770 - val_accuracy: 0.8830
Epoch 8/30
931/931 [==============================] - 120s 128ms/step - loss: 0.3694 - accuracy: 0.8702 - val_loss: 0.3678 - val_accuracy: 0.8908
Epoch 9/30
931/931 [==============================] - 118s 126ms/step - loss: 0.3697 - accuracy: 0.8698 - val_loss: 0.3855 - val_accuracy: 0.8856
Model: "functional_91"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_46 (InputLayer)        [(None, 120, 10)]         0         
_________________________________________________________________
gru_25 (GRU)                 (None, 8)                 480       
_________________________________________________________________
dense_45 (Dense)             (None, 3)                 27        
Total params: 507
Trainable params: 507
Non-trainable params: 0
________________________

Epoch 4/30
931/931 [==============================] - 66s 71ms/step - loss: 0.4709 - accuracy: 0.8205 - val_loss: 0.4959 - val_accuracy: 0.8599
Epoch 5/30
931/931 [==============================] - 65s 70ms/step - loss: 0.4583 - accuracy: 0.8290 - val_loss: 0.5103 - val_accuracy: 0.8556
Epoch 6/30
931/931 [==============================] - 66s 71ms/step - loss: 0.4487 - accuracy: 0.8371 - val_loss: 0.5373 - val_accuracy: 0.8522
Epoch 7/30
931/931 [==============================] - 62s 67ms/step - loss: 0.4399 - accuracy: 0.8416 - val_loss: 0.5342 - val_accuracy: 0.8554
Model: "functional_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_51 (InputLayer)        [(None, 120, 10)]         0         
_________________________________________________________________
gru_30 (GRU)                 (None, 120, 16)           1344      
_________________________________________________________________
gru_

Epoch 1/30
931/931 [==============================] - 254s 272ms/step - loss: 0.4589 - accuracy: 0.8210 - val_loss: 0.4468 - val_accuracy: 0.8401
Epoch 2/30
931/931 [==============================] - 278s 299ms/step - loss: 0.3496 - accuracy: 0.8640 - val_loss: 0.4981 - val_accuracy: 0.8522
Epoch 3/30
931/931 [==============================] - 279s 300ms/step - loss: 0.3083 - accuracy: 0.8820 - val_loss: 0.5565 - val_accuracy: 0.8740
Epoch 4/30
931/931 [==============================] - 265s 285ms/step - loss: 0.2628 - accuracy: 0.9038 - val_loss: 0.6378 - val_accuracy: 0.8603
Epoch 5/30
931/931 [==============================] - 262s 282ms/step - loss: 0.2386 - accuracy: 0.9107 - val_loss: 0.6923 - val_accuracy: 0.8685
Model: "functional_111"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        [(None, 120, 10)]         0         
________________________________________________

931/931 [==============================] - 147s 158ms/step - loss: 0.4512 - accuracy: 0.8210 - val_loss: 0.4502 - val_accuracy: 0.8138
Epoch 3/30
931/931 [==============================] - 145s 156ms/step - loss: 0.4501 - accuracy: 0.8283 - val_loss: 0.4405 - val_accuracy: 0.8116
Epoch 4/30
931/931 [==============================] - 135s 145ms/step - loss: 0.4073 - accuracy: 0.8487 - val_loss: 0.4688 - val_accuracy: 0.8232
Epoch 5/30
931/931 [==============================] - 126s 135ms/step - loss: 0.3729 - accuracy: 0.8639 - val_loss: 0.4965 - val_accuracy: 0.8204
Epoch 6/30
931/931 [==============================] - 145s 156ms/step - loss: 0.3542 - accuracy: 0.8711 - val_loss: 0.5340 - val_accuracy: 0.8145
Epoch 7/30
931/931 [==============================] - 145s 156ms/step - loss: 0.3326 - accuracy: 0.8783 - val_loss: 0.5510 - val_accuracy: 0.8275


In [13]:
for i,h in enumerate(histories):
    np.save('{}.npy'.format(i),h.history)